In [ ]:
#Imports 
import icepyx as ipx
from earthdata import Auth, DataCollections, DataGranules, Store # Used to ingest data from NSIDC
import geopandas as gpd # Used to subset regions using shapefile
import rioxarray as rx # Used to read raster data from hdf5 files
import numpy as np
import matplotlib.pyplot as plt

#%matplotlib inline
%matplotlib widget 

### Set working directories 

In [ ]:
# Set working directories (local)
data_loc = "/home/jovyan/data/"

In [ ]:
# Authenticate Earthdata credentials

auth = Auth().login(strategy='netrc')
if auth.authenticated is False:
    auth = Auth().login(strategy='interactive')

## PART 1: Download ATL15 data in study area basin 

### Step 1: Define all basins 

In [ ]:
#Read in MEaSUREs ice boundaries 
bd = gpd.read_file(data_loc+"/boundaries/IceBoundaries_Antarctica_v02.shp")

In [ ]:
# Plot shapefile to make sure it as expected
bd.plot()

# 'Geometry' can beextracted - here, we print out the head of the geometry field
print(bd['geometry'].head())

### Step 2: Isolate study basin 

In [ ]:
#Set to name of basin 
name = 'Totten'

In [ ]:
# Isolated only grounded ice (GR) for the Totten basin
basin = bd[(bd["NAME"] == name) & (bd["TYPE"] == "GR")]

# Plot the Totten subset
basin.plot()

#Get useful info
#tot.geometry
#tot.crs

In [ ]:
# Identify the bounding box for the given region: total_bounds = total bounding box of all polygons (if multiple); bounds = bounding box for each polygon (if multiple).
# When only one polygon (in this case), .total_bounds, or .bounds are comparable
bbox = tot.total_bounds
bbox

### Step 3: Reproject to EPSG 4326 and get basin coordinates

In [ ]:
#Function to reproject GPD polygon (EPSG 4326 needed for icepyx) 
def reproject(geom, epsg="EPSG:4326"):
    return geom.to_crs(epsg)

In [ ]:
#Reproject basin from 3031 to 4326 
basin_proj = reproject(basin)
basin_proj.plot()

### Step 4: Query ATL15 data in Study Basin

#### Option A) Using coordinates of Polygon 

In [ ]:
#Get basin coordinates 

#Function to extract coordinates from GPD polyon 
def get_coords(geom):
    coords = list(geom.exterior.coords)
    return list(coords)

#extract coordinates of basin polygon 
coords = list(basin_proj.geometry.apply(get_coords))[0]

In [ ]:
#Set inputs to Icepyx download query 
short_name = 'ATL15'
spatial_extent = coords #coordinates from basin (extracted in step 3)
date_range = ['2019-02-20','2022-02-28']

region_a = ipx.Query(short_name, spatial_extent, date_range)
#region_a.visualize_spatial_extent()

#### Option B) Using shapefile

In [ ]:
#Esport geopandas polygon of basin to shapefile 
shp_out = data_loc + f"boundaries/{name}_boundary_4326.shp"
basin_proj.to_file(shp_out)

In [ ]:
#Set inputs to Icepyx download query 
short_name = 'ATL15'
spatial_extent = shp_out #set file path to shapefile (in epsg:4326)
date_range = ['2019-02-20','2022-02-28']

region_a = ipx.Query(short_name, spatial_extent, date_range)
#region_a.visualize_spatial_extent()

In [ ]:
#Print other info
print(region_a.product)
print(region_a.dates)
print(region_a.start_time)
print(region_a.end_time)
print(region_a.cycles)
print(region_a.tracks)
print(region_a.product_version)
#region_a.visualize_spatial_extent()

In [ ]:
region_a.product_summary_info()
print(region_a.latest_version())

### Step 5: Order and Download ATL15 Data

In [ ]:
#Authenticate EarthData Login
earthdata_uid = 'freer_bi.97'
email = 'bryony.freer@gmail.com'
region_a.earthdata_login(earthdata_uid, email)

In [ ]:
#search for available granules and provide basic summary info about them
region_a.avail_granules()

In [ ]:
#order granules 
region_a.order_granules()

In [ ]:
#Download 
path = data_loc + f'/ATL15/{basin}'
#region_a.download_granules(path) 

## PART 2: Plot ATL15 Data 

*Currently work in progress (as of 5pm UTC, 23/03)*

In [ ]:
#opens raster dataset
ds=rx.open_rasterio(f'{path}/ATL15_AA_0311_10km_001_01.nc', group='delta_h', masked=True)
ds

The datasets here represent surface-height differences relative to 2020.0.  For a quick view of what is going on, we can make a map of the mean height change rate from the start of the mission to the present:

In [ ]:
# note that time is in days after Jan 1 2018
dhdt = (ds['delta_h'][-1,:,:]-ds['delta_h'][0,:,:])/(ds['time'][-1]-ds['time'][0])*365.25 #convert the time into units of years (so can get height change in m/yr)
#caluclate extent of dataset for matplotlib
extent=np.array([np.min(ds['x'])-500, np.max(ds['x'])+500, np.min(ds['y'])-500, np.max(ds['y'])+500])

In [ ]:
#maps height change from april 2019 to end of 2020 
plt.figure(); 
hax=plt.gca()
h_im=hax.imshow(dhdt, cmap='Spectral',  extent=extent)
hax.set_aspect(1)
#hax.set_ylim([-0.8e6, -0.2e6])
hax.set_xlabel('polar-stereographic x')
hax.set_ylabel('polar-stereographic y')
plt.colorbar(h_im, label='height-change rate, m/yr');